#### ADVANCED SEMANTIC CHUNKING - BASIC WITH GENSIM (LDA) 
Pada metode ini hanya menggunakan metode chungking berdasarkan topik menggunakan Gensim (LDA - Latent Dirichlet Allocation). Gensim menggunakan model berbasis bag-of-words seperti LDA (Latent Dirichlet Allocation) untuk mengekstrak topik dari teks.

Cara Kerja:
- Memetakan dokumen ke dalam ruang vektor berdasarkan frekuensi kata.
- Menggunakan probabilitas untuk menemukan distribusi kata dalam berbagai topik.

Kelebihan:
- Cocok untuk analisis topik berbasis statistik.
- Tidak memerlukan model berbasis pembelajaran mendalam.

Kekurangan:
- Tidak mempertimbangkan konteks urutan kata dalam kalimat.
- Tidak menghasilkan representasi teks yang dapat digunakan untuk perbandingan semantik.

In [ ]:
import os  # Mengimpor modul os untuk operasi sistem file
import spacy  # Mengimpor spaCy untuk pemrosesan bahasa alami
import PyPDF2  # Mengimpor PyPDF2 untuk membaca file PDF
import docx  # Mengimpor modul docx untuk membaca file DOCX
import gensim  # Mengimpor gensim untuk model topik
from gensim import corpora  # Mengimpor corpora dari gensim untuk membuat kamus

# Memuat model bahasa Inggris kecil dari spaCy
nlp = spacy.load("en_core_web_sm")

def read_txt(file_path):
    # Membaca file teks dan mengembalikan isinya
    with open(file_path, "r", encoding="utf-8") as f:  # Membuka file dalam mode baca
        return f.read()  # Mengembalikan seluruh isi file

def read_pdf(file_path):
    # Membaca file PDF dan mengembalikan isinya
    text = ""  # Inisialisasi string kosong untuk menyimpan teks
    with open(file_path, "rb") as f:  # Membuka file PDF dalam mode biner
        reader = PyPDF2.PdfReader(f)  # Membaca file PDF
        for page in reader.pages:  # Iterasi melalui setiap halaman
            text += page.extract_text() + "\n"  # Ekstrak teks dari halaman dan tambahkan ke string
    return text  # Mengembalikan teks yang diekstrak

def read_docx(file_path):
    # Membaca file DOCX dan mengembalikan isinya
    doc = docx.Document(file_path)  # Membuka file DOCX
    return "\n".join([para.text for para in doc.paragraphs])  # Menggabungkan teks dari setiap paragraf

def extract_topics(text, num_topics=3):
    # Mengekstrak topik dari teks menggunakan model LDA
    words = [token.lemma_ for token in nlp(text) if token.is_alpha and not token.is_stop]  # Tokenisasi dan lemmatization

    if not words:  # Jika tidak ada kata yang valid
        return ["[No valid words for topic extraction]"]  # Mengembalikan pesan bahwa tidak ada kata valid

    dictionary = corpora.Dictionary([words])  # Membuat kamus dari kata-kata
    corpus = [dictionary.doc2bow(words)]  # Membuat korpus dari kata-kata

    if not corpus or all(len(doc) == 0 for doc in corpus):  # Jika korpus kosong
        return ["[No significant topics]"]  # Mengembalikan pesan bahwa tidak ada topik signifikan

    # Membuat model LDA untuk mengekstrak topik
    lda_model = gensim.models.LdaModel(corpus, num_topics=min(num_topics, len(dictionary)), 
    id2word=dictionary, passes=10)
    topics = lda_model.print_topics()  # Mencetak topik yang diekstrak

    return topics  # Mengembalikan daftar topik

def advanced_chunk_text(text, chunk_size=700):
    # Membagi teks menjadi bagian-bagian kecil berdasarkan ukuran dan entitas
    doc = nlp(text)  # Memproses teks dengan spaCy
    chunks = []  # Inisialisasi daftar untuk menyimpan bagian
    current_chunk = ""  # Inisialisasi string untuk bagian saat ini
    current_entities = set()  # Inisialisasi set untuk menyimpan entitas saat ini
    
    for sent in doc.sents:  # Iterasi melalui setiap kalimat
        entities = {ent.text for ent in sent.ents}  # Mengambil entitas dari kalimat
        # Jika ukuran bagian saat ini ditambah kalimat masih dalam batas chunk_size
        if len(current_chunk) + len(sent.text) < chunk_size and (not current_entities or entities & current_entities):
            current_chunk += " " + sent.text  # Tambahkan kalimat ke bagian saat ini
            current_entities.update(entities)  # Perbarui entitas saat ini
        else:
            chunks.append(current_chunk.strip())  # Tambahkan bagian saat ini ke daftar
            current_chunk = sent.text  # Mulai bagian baru dengan kalimat saat ini
            current_entities = entities  # Perbarui entitas saat ini
    
    if current_chunk:  # Jika ada bagian yang tersisa
        chunks.append(current_chunk.strip())  # Tambahkan bagian terakhir ke daftar
    
    return chunks  # Mengembalikan daftar bagian

def process_document(file_path, output_folder):
    # Memproses dokumen berdasarkan jenis file dan menyimpan hasilnya
    ext = file_path.split(".")[-1].lower()  # Mendapatkan ekstensi file
    
    # Membaca teks berdasarkan jenis file
    if ext == "txt":
        text = read_txt(file_path)  # Membaca file teks
    elif ext == "pdf":
        text = read_pdf (file_path)  # Membaca file PDF
    elif ext == "docx":
        text = read_docx(file_path)  # Membaca file DOCX
    else:
        return  # Jika jenis file tidak dikenali, keluar dari fungsi
    
    base_name = os.path.basename(file_path).split(".")[0]  # Mendapatkan nama dasar file tanpa ekstensi
    file_output_folder = os.path.join(output_folder, base_name)  # Menentukan folder output untuk file ini
    os.makedirs(file_output_folder, exist_ok=True)  # Membuat folder output jika belum ada
    
    chunks = advanced_chunk_text(text)  # Membagi teks menjadi bagian-bagian kecil
    
    for i, chunk in enumerate(chunks):  # Iterasi melalui setiap bagian
        chunk_file = os.path.join(file_output_folder, f"{base_name}_chunk_{i+1}.txt")  # Menentukan nama file untuk bagian
        topics = extract_topics(chunk)  # Mengekstrak topik dari bagian
        
        with open(chunk_file, "w", encoding="utf-8") as f:  # Membuka file untuk menulis
            f.write(chunk + "\n\n")  # Menulis bagian ke file
            f.write("Topik: \n")  # Menulis header untuk topik
            for topic in topics:  # Iterasi melalui setiap topik
                f.write(f"{topic}\n")  # Menulis topik ke file
    
    print(f"Processed: {file_path} -> {file_output_folder}")  # Menampilkan pesan bahwa file telah diproses

def main():
    # Fungsi utama untuk menjalankan proses
    input_folder = "data"  # Menentukan folder input
    output_folder = "output/basic-gensim"  # Menentukan folder output
    os.makedirs(output_folder, exist_ok=True)  # Membuat folder output jika belum ada
    
    for file_name in os.listdir(input_folder):  # Iterasi melalui setiap file dalam folder input
        file_path = os.path.join(input_folder, file_name)  # Mendapatkan jalur lengkap file
        if os.path.isfile(file_path):  # Memastikan bahwa jalur adalah file
            process_document(file_path, output_folder)  # Memproses dokumen

if __name__ == "__main__":
    main()  # Menjalankan fungsi utama saat skrip dieksekusi

Processed: data\dokumen_pdf.pdf -> output/basic-gensim\dokumen_pdf
Processed: data\tko.pdf -> output/basic-gensim\tko


#### ADVANCED SEMANTIC CHUNKING - BASIC WITH SENTENCETRANSFORMER (BERT)
Pada metode ini hanya menggunakan metode chungking berdasarkan topik menggunakan SentenceTransformer (BERT-based Embeddings). SentenceTransformer menggunakan model berbasis Transformer (BERT, RoBERTa, dll.) untuk menghasilkan embedding kalimat yang lebih kontekstual.

Cara Kerja:
- Mengubah kalimat menjadi vektor berdimensi tinggi menggunakan model deep learning.
- Mempertimbangkan makna keseluruhan teks dalam konteksnya.

Kelebihan:
- Menghasilkan embedding yang lebih kaya dan kontekstual.
- Bisa digunakan untuk perbandingan kemiripan antar-kalimat.

Kekurangan:
- Memerlukan lebih banyak daya komputasi dibandingkan dengan Gensim LDA.
- Model lebih besar dan memerlukan dependensi tambahan seperti PyTorch atau TensorFlow.

In [10]:
import os  # Mengimpor modul os untuk operasi sistem file
import spacy  # Mengimpor spaCy untuk pemrosesan bahasa alami
import PyPDF2  # Mengimpor PyPDF2 untuk membaca file PDF
import docx  # Mengimpor modul docx untuk membaca file DOCX
from sentence_transformers import SentenceTransformer  # Mengimpor SentenceTransformer untuk model embedding kalimat

def read_txt(file_path):
    # Membaca file teks dan mengembalikan isinya
    with open(file_path, "r", encoding="utf-8") as f:  # Membuka file dalam mode baca
        return f.read()  # Mengembalikan seluruh isi file

def read_pdf(file_path):
    # Membaca file PDF dan mengembalikan isinya
    text = ""  # Inisialisasi string kosong untuk menyimpan teks
    with open(file_path, "rb") as f:  # Membuka file PDF dalam mode biner
        reader = PyPDF2.PdfReader(f)  # Membaca file PDF
        for page in reader.pages:  # Iterasi melalui setiap halaman
            text += page.extract_text() + "\n"  # Ekstrak teks dari halaman dan tambahkan ke string
    return text  # Mengembalikan teks yang diekstrak

def read_docx(file_path):
    # Membaca file DOCX dan mengembalikan isinya
    doc = docx.Document(file_path)  # Membuka file DOCX
    return "\n".join([para.text for para in doc.paragraphs])  # Menggabungkan teks dari setiap paragraf

# Memuat model bahasa Inggris kecil dari spaCy
nlp = spacy.load("en_core_web_sm")
# Memuat model SentenceTransformer untuk embedding kalimat
model = SentenceTransformer("all-MiniLM-L6-v2")

def extract_topics(text, num_topics=5):
    # Mengekstrak topik dari teks menggunakan model SentenceTransformer
    sentences = [sent.text for sent in nlp(text).sents]  # Memecah teks menjadi kalimat

    if not sentences:  # Jika tidak ada kalimat yang valid
        return ["[No valid sentences for topic extraction]"]  # Mengembalikan pesan bahwa tidak ada kalimat valid

    embeddings = model.encode(sentences)  # Menghitung embedding untuk setiap kalimat
    return embeddings[:num_topics]  # Mengembalikan embedding untuk jumlah topik yang diminta

def advanced_chunk_text(text, chunk_size=700):
    # Membagi teks menjadi bagian-bagian kecil berdasarkan ukuran dan entitas
    doc = nlp(text)  # Memproses teks dengan spaCy
    chunks = []  # Inisialisasi daftar untuk menyimpan bagian
    current_chunk = ""  # Inisialisasi string untuk bagian saat ini
    current_entities = set()  # Inisialisasi set untuk menyimpan entitas saat ini
    
    for sent in doc.sents:  # Iterasi melalui setiap kalimat
        entities = {ent.text for ent in sent.ents}  # Mengambil entitas dari kalimat
        # Jika ukuran bagian saat ini ditambah kalimat masih dalam batas chunk_size
        if len(current_chunk) + len(sent.text) < chunk_size and (not current_entities or entities & current_entities):
            current_chunk += " " + sent.text  # Tambahkan kalimat ke bagian saat ini
            current_entities.update(entities)  # Perbarui entitas saat ini
        else:
            chunks.append(current_chunk.strip())  # Tambahkan bagian saat ini ke daftar
            current_chunk = sent.text  # Mulai bagian baru dengan kalimat saat ini
            current_entities = entities  # Perbarui entitas saat ini
    
    if current_chunk:  # Jika ada bagian yang tersisa
        chunks.append(current_chunk.strip())  # Tambahkan bagian terakhir ke daftar
    
    return chunks  # Mengembalikan daftar bagian

def process_document(file_path, output_folder):
    # Memproses dokumen berdasarkan jenis file dan menyimpan hasilnya
    ext = file_path.split(".")[-1].lower()  # Mendapatkan ekstensi file
    
    # Membaca teks berdasarkan jenis file
    if ext == "txt":
        text = read_txt(file_path)  # Membaca file teks
    elif ext == "pdf":
        text = read_pdf(file_path)  # Membaca file PDF
    elif ext == "docx":
        text = read_docx(file_path)  # Membaca file DOCX
    else:
        return  # Jika jenis file tidak dikenali, keluar dari fungsi
    
    base_name = os.path.basename(file_path).split(".")[0]  # Mendapatkan nama dasar file tanpa ekstensi
    file_output_folder = os.path.join(output_folder, base_name)  # Menentukan folder output untuk file ini
    os.makedirs(file_output_folder, exist_ok=True)  # Membuat folder output jika belum ada
    
    chunks = advanced_chunk_text(text)  # Membagi teks menjadi bagian-bagian kecil
    
    for i, chunk in enumerate(chunks):  # Iterasi melalui setiap bagian
        chunk_file = os.path.join(file_output_folder, f"{base_name}_chunk_{i+1}.txt")  # Menentukan nama file untuk bagian
        topics = extract_topics(chunk)  # Mengekstrak topik dari bagian

        with open(chunk_file, "w", encoding="utf-8") as f:  # Membuka file untuk menulis
            f.write(chunk + "\n\n")  # Menulis bagian ke file
            f.write("Topik:\n")  # Menulis header untuk topik
            for topic in topics:  # Iterasi melalui setiap topik
                f.write(f"{topic}\n")  # Menulis topik ke file
    
    print(f"Processed: {file_path} -> {file_output_folder}")  # Menampilkan pesan bahwa file telah diproses

def main():
    # Fungsi utama untuk menjalankan proses
    input_folder = "data"  # Menentukan folder input
    output_folder = "output/basic-sentence-transformers"  # Menentukan folder output
    os.makedirs(output_folder, exist_ok=True)  # Membuat folder output jika belum ada
    
    for file_name in os.listdir(input_folder):  # Iterasi melalui setiap file dalam folder input
        file_path = os.path.join(input_folder, file_name)  # Mendapatkan jalur lengkap file
        if os.path.isfile(file_path):  # Memastikan bahwa jalur adalah file
            process_document(file_path, output_folder)  # Memproses dokumen

if __name__ == "__main__":
    main()  # Menjalankan fungsi utama saat skrip dieksekusi

Processed: data\dokumen_pdf.pdf -> output/basic-sentence-transformers\dokumen_pdf
Processed: data\tko.pdf -> output/basic-sentence-transformers\tko


#### ADVANCED SEMANTIC CHUNKING - NODE ONLY

Metode ini hanya menggunakan teknik chunking berbasis graf tanpa perangkingan. Model ini membentuk node berdasarkan kemiripan semantik antar kalimat menggunakan SentenceTransformer dan graph-based clustering.

Cara Kerja:
- Teks dipecah menjadi kalimat.
- Kalimat direpresentasikan sebagai vektor embedding menggunakan SentenceTransformer.
- Graf dibentuk berdasarkan cosine similarity antar kalimat.
- Komunitas dalam graf ditentukan menggunakan greedy modularity optimization, yang menghasilkan chunk berbasis hubungan semantik.

Kelebihan:
- Memanfaatkan hubungan semantik antar kalimat.
- Cocok untuk dokumen dengan struktur naratif yang kuat.

Kekurangan:
- Tidak ada prioritas dalam hasil chunking.
- Semua chunk dianggap memiliki bobot yang sama dalam informasi.

In [ ]:
import os  # Mengimpor modul os untuk operasi sistem file
import spacy  # Mengimpor spaCy untuk pemrosesan bahasa alami
import docx  # Mengimpor modul docx untuk membaca file DOCX
import PyPDF2  # Mengimpor PyPDF2 untuk membaca file PDF
import networkx as nx  # Mengimpor NetworkX untuk analisis graf
import gensim  # Mengimpor gensim untuk model topik
from gensim import corpora  # Mengimpor corpora dari gensim untuk membuat kamus
from sklearn.metrics.pairwise import cosine_similarity  # Mengimpor fungsi untuk menghitung kesamaan kosinus
from sentence_transformers import SentenceTransformer  # Mengimpor SentenceTransformer untuk model embedding kalimat

# Memuat model bahasa Inggris kecil dari spaCy
nlp = spacy.load("en_core_web_sm")
# Memuat model SentenceTransformer untuk embedding kalimat
model = SentenceTransformer("all-MiniLM-L6-v2")

def read_txt(file_path):
    # Membaca file teks dan mengembalikan isinya
    with open(file_path, "r", encoding="utf-8") as f:  # Membuka file dalam mode baca
        return f.read()  # Mengembalikan seluruh isi file

def read_pdf(file_path):
    # Membaca file PDF dan mengembalikan isinya
    text = ""  # Inisialisasi string kosong untuk menyimpan teks
    with open(file_path, "rb") as f:  # Membuka file PDF dalam mode biner
        reader = PyPDF2.PdfReader(f)  # Membaca file PDF
        for page in reader.pages:  # Iterasi melalui setiap halaman
            text += page.extract_text() + "\n"  # Ekstrak teks dari halaman dan tambahkan ke string
    return text  # Mengembalikan teks yang diekstrak

def read_docx(file_path):
    # Membaca file DOCX dan mengembalikan isinya
    doc = docx.Document(file_path)  # Membuka file DOCX
    return "\n".join([para.text for para in doc.paragraphs])  # Menggabungkan teks dari setiap paragraf

def graph_based_chunking(text, chunk_size=700):
    # Membagi teks menjadi bagian-bagian kecil menggunakan pendekatan berbasis graf
    doc = nlp(text)  # Memproses teks dengan spaCy
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]  # Mengambil kalimat yang tidak kosong
    
    if not sentences:  # Jika tidak ada kalimat yang valid
        return []  # Mengembalikan daftar kosong
    
    sentence_embeddings = model.encode(sentences)  # Menghitung embedding untuk setiap kalimat
    similarity_matrix = cosine_similarity(sentence_embeddings)  # Menghitung matriks kesamaan kosinus
    G = nx.Graph()  # Membuat graf kosong
    
    # Menambahkan tepi ke graf berdasarkan kesamaan kalimat
    for i in range(len(sentences)):
        for j in range(i + 1, len(sentences)):
            G.add_edge(i, j, weight=similarity_matrix[i, j])  # Menambahkan tepi dengan bobot kesamaan
    
    # Menggunakan algoritma komunitas untuk menemukan partisi dalam graf
    partitions = nx.community.greedy_modularity_communities(G)
    
    chunks = []  # Inisialisasi daftar untuk menyimpan bagian
    for community in partitions:  # Iterasi melalui setiap komunitas
        chunk = " ".join([sentences[i] for i in sorted(community)])  # Menggabungkan kalimat dalam komunitas menjadi satu bagian
        if len(chunk) > chunk_size:  # Jika ukuran bagian melebihi chunk_size
            # Membagi bagian menjadi sub-bagian yang lebih kecil
            sub_chunks = [chunk[i:i+chunk_size] for i in range(0, len(chunk), chunk_size)]
            chunks.extend(sub_chunks)  # Menambahkan sub-bagian ke daftar
        else:
            chunks.append(chunk)  # Menambahkan bagian ke daftar
    
    return chunks  # Mengembalikan daftar bagian

def extract_topics(text, num_topics=3):
    # Mengekstrak topik dari teks menggunakan model LDA
    words = [token.lemma_ for token in nlp(text) if token.is_alpha and not token.is_stop]  # Tokenisasi dan lemmatization

    if not words:  # Jika tidak ada kata yang valid
        return ["[No valid words for topic extraction]"]  # Mengembalikan pesan bahwa tidak ada kata valid

    dictionary = corpora.Dictionary([words])  # Membuat kamus dari kata-kata
    corpus = [dictionary.doc2bow(words)]  # Membuat korpus dari kata-kata

    if not corpus or all(len(doc) == 0 for doc in corpus):  # Jika korpus kosong
        return ["[No significant topics ]"]  # Mengembalikan pesan bahwa tidak ada topik yang signifikan

    lda_model = gensim.models.LdaModel(corpus, num_topics=min(num_topics, len(dictionary)), 
    id2word=dictionary, passes=10)  # Membuat model LDA
    topics = lda_model.print_topics()  # Mengambil topik yang dihasilkan oleh model

    return topics  # Mengembalikan daftar topik

def process_document(file_path, output_folder):
    # Memproses dokumen berdasarkan jenis file dan menyimpan hasilnya
    ext = file_path.split(".")[-1].lower()  # Mendapatkan ekstensi file
    
    # Membaca teks berdasarkan jenis file
    if ext == "txt":
        text = read_txt(file_path)  # Membaca file teks
    elif ext == "pdf":
        text = read_pdf(file_path)  # Membaca file PDF
    elif ext == "docx":
        text = read_docx(file_path)  # Membaca file DOCX
    else:
        return  # Jika jenis file tidak dikenali, keluar dari fungsi
    
    base_name = os.path.basename(file_path).split(".")[0]  # Mendapatkan nama dasar file tanpa ekstensi
    file_output_folder = os.path.join(output_folder, base_name)  # Menentukan folder output untuk file ini
    os.makedirs(file_output_folder, exist_ok=True)  # Membuat folder output jika belum ada
    
    chunks = graph_based_chunking(text)  # Membagi teks menjadi bagian-bagian kecil

    for i, chunk in enumerate(chunks):  # Iterasi melalui setiap bagian
        chunk_file = os.path.join(file_output_folder, f"{base_name}_chunk_{i+1}.txt")  # Menentukan nama file untuk bagian
        topics = extract_topics(chunk)  # Mengekstrak topik dari bagian
        
        with open(chunk_file, "w", encoding="utf-8") as f:  # Membuka file untuk menulis
            f.write(chunk + "\n\n")  # Menulis bagian ke file
            f.write("=== Topics ===\n")  # Menulis header untuk topik
            for topic in topics:  # Iterasi melalui setiap topik
                f.write(f"{topic}\n")  # Menulis topik ke file

    print(f"Processed: {file_path} -> {file_output_folder}")  # Menampilkan pesan bahwa file telah diproses

def main():
    # Fungsi utama untuk menjalankan proses
    input_folder = "data"  # Menentukan folder input
    output_folder = "output/graph-node-only"  # Menentukan folder output
    os.makedirs(output_folder, exist_ok=True)  # Membuat folder output jika belum ada
    
    for file_name in os.listdir(input_folder):  # Iterasi melalui setiap file dalam folder input
        file_path = os.path.join(input_folder, file_name)  # Mendapatkan jalur lengkap file
        if os.path.isfile(file_path):  # Memastikan bahwa jalur adalah file
            process_document(file_path, output_folder)  # Memproses dokumen

if __name__ == "__main__":
    main()  # Menjalankan fungsi utama saat skrip dieksekusi

Processed: data\dokumen_pdf.pdf -> output/graph-node-only\dokumen_pdf
Processed: data\tko.pdf -> output/graph-node-only\tko


#### ADVANCED SEMANTIC CHUNKING - NODE WITH CHUNK RANK

Metode ini menggunakan chunking berbasis graf, namun dengan tambahan perangkingan chunk berdasarkan jumlah kata atau kepadatan informasi.

Cara Kerja:
- Proses chunking sama seperti metode "Node Only".
- Setelah chunk terbentuk, setiap chunk diberi skor berdasarkan jumlah kata.
- Chunk dengan informasi lebih padat diberikan peringkat lebih tinggi.

Kelebihan:
- Memungkinkan ekstraksi chunk yang lebih informatif.
- Cocok untuk proses summarization berbasis chunk.

Kekurangan:
- Ranking berdasarkan jumlah kata belum tentu mencerminkan makna semantik yang lebih penting.
- Bisa menyebabkan perubahan urutan asli dokumen jika digunakan untuk reordering.

In [12]:
import os  # Mengimpor modul os untuk operasi sistem file
import spacy  # Mengimpor spaCy untuk pemrosesan bahasa alami
import docx  # Mengimpor modul docx untuk membaca file DOCX
import PyPDF2  # Mengimpor PyPDF2 untuk membaca file PDF
import networkx as nx  # Mengimpor NetworkX untuk analisis graf
import gensim  # Mengimpor gensim untuk model topik
import numpy as np  # Mengimpor numpy untuk operasi numerik
from gensim import corpora  # Mengimpor corpora dari gensim untuk membuat kamus
from sklearn.metrics.pairwise import cosine_similarity  # Mengimpor fungsi untuk menghitung kesamaan kosinus
from sentence_transformers import SentenceTransformer  # Mengimpor SentenceTransformer untuk model embedding kalimat

# Memuat model bahasa Inggris kecil dari spaCy
nlp = spacy.load("en_core_web_sm")
# Memuat model SentenceTransformer untuk embedding kalimat
model = SentenceTransformer("all-MiniLM-L6-v2")

def read_txt(file_path):
    # Membaca file teks dan mengembalikan isinya
    with open(file_path, "r", encoding="utf-8") as f:  # Membuka file dalam mode baca
        return f.read()  # Mengembalikan seluruh isi file

def read_pdf(file_path):
    # Membaca file PDF dan mengembalikan isinya
    text = ""  # Inisialisasi string kosong untuk menyimpan teks
    with open(file_path, "rb") as f:  # Membuka file PDF dalam mode biner
        reader = PyPDF2.PdfReader(f)  # Membaca file PDF
        for page in reader.pages:  # Iterasi melalui setiap halaman
            text += page.extract_text() + "\n"  # Ekstrak teks dari halaman dan tambahkan ke string
    return text  # Mengembalikan teks yang diekstrak

def read_docx(file_path):
    # Membaca file DOCX dan mengembalikan isinya
    doc = docx.Document(file_path)  # Membuka file DOCX
    return "\n".join([para.text for para in doc.paragraphs])  # Menggabungkan teks dari setiap paragraf

def graph_based_chunking(text, chunk_size=700):
    # Membagi teks menjadi bagian-bagian kecil menggunakan pendekatan berbasis graf
    doc = nlp(text)  # Memproses teks dengan spaCy
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]  # Mengambil kalimat yang tidak kosong
    
    if not sentences:  # Jika tidak ada kalimat yang valid
        return []  # Mengembalikan daftar kosong
    
    sentence_embeddings = model.encode(sentences)  # Menghitung embedding untuk setiap kalimat
    similarity_matrix = cosine_similarity(sentence_embeddings)  # Menghitung matriks kesamaan kosinus
    G = nx.Graph()  # Membuat graf kosong
    
    # Menambahkan tepi ke graf berdasarkan kesamaan kalimat
    for i in range(len(sentences)):
        for j in range(i + 1, len(sentences)):
            G.add_edge(i, j, weight=similarity_matrix[i, j])  # Menambahkan tepi dengan bobot kesamaan
    
    # Menggunakan algoritma komunitas untuk menemukan partisi dalam graf
    partitions = nx.community.greedy_modularity_communities(G)
    
    chunks = []  # Inisialisasi daftar untuk menyimpan bagian
    for community in partitions:  # Iterasi melalui setiap komunitas
        chunk = " ".join([sentences[i] for i in sorted(community)])  # Menggabungkan kalimat dalam komunitas menjadi satu bagian
        if len(chunk) > chunk_size:  # Jika ukuran bagian melebihi chunk_size
            # Membagi bagian menjadi sub-bagian yang lebih kecil
            sub_chunks = [chunk[i:i+chunk_size] for i in range(0, len(chunk), chunk_size)]
            chunks.extend(sub_chunks)  # Menambahkan sub-bagian ke daftar
        else:
            chunks.append(chunk)  # Menambahkan bagian ke daftar
    
    return chunks  # Mengembalikan daftar bagian

def extract_topics(text, num_topics=3):
    # Mengekstrak topik dari teks menggunakan model LDA
    words = [token.lemma_ for token in nlp(text) if token.is_alpha and not token.is_stop]  # Tokenisasi dan lemmatization

    if not words:  # Jika tidak ada kata yang valid
        return ["[No valid words for topic extraction]"]  # Mengembalikan pesan bahwa tidak ada kata valid

    dictionary = corpora.Dictionary([words])  # Membuat kamus dari kata-kata
    corpus = [dictionary.doc2bow(words)]  # Membuat korpus dari kata-kata

    if not corpus or all(len(doc) == 0 for doc in corpus ):  # Jika korpus kosong
        return ["[No significant topics]"]  # Mengembalikan pesan bahwa tidak ada topik yang signifikan

    lda_model = gensim.models.LdaModel(corpus, num_topics=min(num_topics, len(dictionary)), 
    id2word=dictionary, passes=10)  # Membuat model LDA
    topics = lda_model.print_topics()  # Mengambil topik yang dihasilkan oleh model

    return topics  # Mengembalikan daftar topik

def rank_chunks(chunks):
    # Mengurutkan bagian berdasarkan jumlah kata dalam urutan menurun
    ranked_chunks = sorted(chunks, key=lambda chunk: len(chunk.split()), reverse=True)  # Mengurutkan bagian
    return ranked_chunks  # Mengembalikan daftar bagian yang terurut

def process_document(file_path, output_folder):
    # Memproses dokumen berdasarkan jenis file dan menyimpan hasilnya
    ext = file_path.split(".")[-1].lower()  # Mendapatkan ekstensi file
    
    # Membaca teks berdasarkan jenis file
    if ext == "txt":
        text = read_txt(file_path)  # Membaca file teks
    elif ext == "pdf":
        text = read_pdf(file_path)  # Membaca file PDF
    elif ext == "docx":
        text = read_docx(file_path)  # Membaca file DOCX
    else:
        return  # Jika jenis file tidak dikenali, keluar dari fungsi
    
    base_name = os.path.basename(file_path).split(".")[0]  # Mendapatkan nama dasar file tanpa ekstensi
    file_output_folder = os.path.join(output_folder, base_name)  # Menentukan folder output untuk file ini
    os.makedirs(file_output_folder, exist_ok=True)  # Membuat folder output jika belum ada
    
    chunks = graph_based_chunking(text)  # Membagi teks menjadi bagian-bagian kecil
    ranked_chunks = rank_chunks(chunks)  # Mengurutkan bagian berdasarkan panjangnya
    
    for i, chunk in enumerate(ranked_chunks):  # Iterasi melalui setiap bagian yang terurut
        chunk_file = os.path.join(file_output_folder, f"{base_name}_chunk_{i+1}.txt")  # Menentukan nama file untuk bagian
        topics = extract_topics(chunk)  # Mengekstrak topik dari bagian
        
        with open(chunk_file, "w", encoding="utf-8") as f:  # Membuka file untuk menulis
            f.write(f"--- Chunk {i+1} (Words: {len(chunk.split())}) ---\n{chunk}\n\n")  # Menulis bagian ke file
            f.write("--- Topics ---\n")  # Menulis header untuk topik
            for topic in topics:  # Iterasi melalui setiap topik
                f.write(f"{topic}\n")  # Menulis topik ke file

    print(f"Processed: {file_path} -> {file_output_folder}")  # Menampilkan pesan bahwa file telah diproses

def main():
    # Fungsi utama untuk menjalankan proses
    input_folder = "data"  # Menentukan folder input
    output_folder = "output/graph-chunk-rank"  # Menentukan folder output
    os.makedirs(output_folder, exist_ok=True)  # Membuat folder output jika belum ada
    
    for file_name in os.listdir(input_folder):  # Iterasi melalui setiap file dalam folder input
        file_path = os.path.join(input_folder, file_name)  # Mendapatkan jalur lengkap file
        if os.path.isfile(file_path):  # Memastikan bahwa jalur adalah file
            process_document(file_path, output_folder)  # Memproses dokumen

if __name__ == "__main__":
    main()  # Menjalankan fungsi utama saat skrip dieksekusi

Processed: data\dokumen_pdf.pdf -> output/graph-chunk-rank\dokumen_pdf
Processed: data\tko.pdf -> output/graph-chunk-rank\tko


#### ADVANCED SEMANTIC CHUNKING - NODE WITH TOPIC RANK

Metode ini menggunakan chunking berbasis graf dan menambahkan perangkingan berbasis topik menggunakan LDA (Latent Dirichlet Allocation).

Cara Kerja:
- Setelah chunk terbentuk, topik utama diekstrak dari setiap chunk menggunakan Gensim LDA.
- Topik terbaik dipilih berdasarkan probabilitas tertinggi dalam distribusi topik chunk tersebut.

Kelebihan:
- Memungkinkan identifikasi topik utama dalam setiap chunk.
- Berguna untuk analisis topik otomatis dari dokumen panjang.

Kekurangan:
- LDA berbasis bag-of-words, sehingga tidak mempertimbangkan urutan kata.
- Hasil topik bisa kurang akurat jika jumlah topik tidak ditentukan dengan baik.

In [13]:
import os  # Mengimpor modul os untuk operasi sistem file
import spacy  # Mengimpor spaCy untuk pemrosesan bahasa alami
import docx  # Mengimpor modul docx untuk membaca file DOCX
import PyPDF2  # Mengimpor PyPDF2 untuk membaca file PDF
import networkx as nx  # Mengimpor NetworkX untuk analisis graf
import gensim  # Mengimpor gensim untuk model topik
from gensim import corpora  # Mengimpor corpora dari gensim untuk membuat kamus
from sklearn.metrics.pairwise import cosine_similarity  # Mengimpor fungsi untuk menghitung kesamaan kosinus
from sentence_transformers import SentenceTransformer  # Mengimpor SentenceTransformer untuk model embedding kalimat

# Memuat model bahasa Inggris kecil dari spaCy
nlp = spacy.load("en_core_web_sm")
# Memuat model SentenceTransformer untuk embedding kalimat
model = SentenceTransformer("all-MiniLM-L6-v2")

def read_txt(file_path):
    # Membaca file teks dan mengembalikan isinya
    with open(file_path, "r", encoding="utf-8") as f:  # Membuka file dalam mode baca
        return f.read()  # Mengembalikan seluruh isi file

def read_pdf(file_path):
    # Membaca file PDF dan mengembalikan isinya
    text = ""  # Inisialisasi string kosong untuk menyimpan teks
    with open(file_path, "rb") as f:  # Membuka file PDF dalam mode biner
        reader = PyPDF2.PdfReader(f)  # Membaca file PDF
        for page in reader.pages:  # Iterasi melalui setiap halaman
            text += page.extract_text() + "\n"  # Ekstrak teks dari halaman dan tambahkan ke string
    return text  # Mengembalikan teks yang diekstrak

def read_docx(file_path):
    # Membaca file DOCX dan mengembalikan isinya
    doc = docx.Document(file_path)  # Membuka file DOCX
    return "\n".join([para.text for para in doc.paragraphs])  # Menggabungkan teks dari setiap paragraf

def graph_based_chunking(text, chunk_size=700):
    # Membagi teks menjadi bagian-bagian kecil menggunakan pendekatan berbasis graf
    doc = nlp(text)  # Memproses teks dengan spaCy
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]  # Mengambil kalimat yang tidak kosong
    
    if not sentences:  # Jika tidak ada kalimat yang valid
        return []  # Mengembalikan daftar kosong
    
    sentence_embeddings = model.encode(sentences)  # Menghitung embedding untuk setiap kalimat
    similarity_matrix = cosine_similarity(sentence_embeddings)  # Menghitung matriks kesamaan kosinus
    G = nx.Graph()  # Membuat graf kosong
    
    # Menambahkan tepi ke graf berdasarkan kesamaan kalimat
    for i in range(len(sentences)):
        for j in range(i + 1, len(sentences)):
            G.add_edge(i, j, weight=similarity_matrix[i, j])  # Menambahkan tepi dengan bobot kesamaan
    
    # Menggunakan algoritma komunitas untuk menemukan partisi dalam graf
    partitions = nx.community.greedy_modularity_communities(G)
    
    chunks = []  # Inisialisasi daftar untuk menyimpan bagian
    for community in partitions:  # Iterasi melalui setiap komunitas
        chunk = " ".join([sentences[i] for i in sorted(community)])  # Menggabungkan kalimat dalam komunitas menjadi satu bagian
        if len(chunk) > chunk_size:  # Jika ukuran bagian melebihi chunk_size
            # Membagi bagian menjadi sub-bagian yang lebih kecil
            sub_chunks = [chunk[i:i+chunk_size] for i in range(0, len(chunk), chunk_size)]
            chunks.extend(sub_chunks)  # Menambahkan sub-bagian ke daftar
        else:
            chunks.append(chunk)  # Menambahkan bagian ke daftar
    
    return chunks  # Mengembalikan daftar bagian

def extract_topics(text, num_topics=3):
    # Mengekstrak topik dari teks menggunakan model LDA
    words = [token.lemma_ for token in nlp(text) if token.is_alpha and not token.is_stop]  # Tokenisasi dan lemmatization

    if not words:  # Jika tidak ada kata yang valid
        return ["No valid words for topic extraction"]  # Mengembalikan pesan bahwa tidak ada kata valid

    dictionary = corpora.Dictionary([words])  # Membuat kamus dari kata-kata
    corpus = [dictionary.doc2bow(words)]  # Membuat korpus dari kata-kata

    if not corpus or all(len(doc) == 0 for doc in corpus):  # Jika korpus kosong
        return ["No significant topics"]  # Mengembalikan pesan bahwa tidak ada topik yang signifikan

    num_topics = min(num_topics, len(dictionary))  # Menentukan jumlah topik yang akan diekstrak
    lda_model = gensim.models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)  # Membuat model LDA
    topics = lda_model.show_topics(formatted=False)  # Mengambil topik yang dihasilkan oleh model

    ranked_topics = sorted(topics, key=lambda x: -sum(prob for _, prob in x[1]))  # Mengurutkan topik berdasarkan probabilitas

    return [f"Topic {t[0]}: {[word for word, _ in t[1]]}" for t in ranked_topics]  # Mengembalikan daftar topik yang terurut

def process_document(file_path, output_folder):
    # Memproses dokumen berdasarkan jenis file dan menyimpan hasilnya
    ext = file_path.split(".")[-1].lower()  # Mendapatkan ekstensi file
    
    # Membaca teks berdasarkan jenis file
    if ext == "txt":
        text = read_txt(file_path)  # Membaca file teks
    elif ext == "pdf":
        text = read_pdf(file_path)  # Membaca file PDF
    elif ext == "docx":
        text = read_docx(file_path)  # Membaca file DOCX
    else:
        return  # Jika jenis file tidak dikenali, keluar dari fungsi
    
    base_name = os.path.basename(file_path).split(".")[0]  # Mendapatkan nama dasar file tanpa ekstensi
    file_output_folder = os.path.join(output_folder, base_name)  # Menentukan folder output untuk file ini
    os.makedirs(file_output_folder, exist_ok=True)  # Membuat folder output jika belum ada
    
    chunks = graph_based_chunking(text)  # Membagi teks menjadi bagian-bagian kecil

    for i, chunk in enumerate(chunks):  # Iterasi melalui setiap bagian
        chunk_file = os.path.join(file_output_folder, f"{base_name}_chunk_{i+1}.txt")  # Menentukan nama file untuk bagian
        best_topics = extract_topics(chunk)  # Mengekstrak topik dari bagian

        with open(chunk_file, "w", encoding="utf-8") as f:  # Membuka file untuk menulis
            f.write(chunk + "\n\nTopics:\n")  # Menulis bagian ke file
            for topic in best_topics:  # Iterasi melalui setiap topik
                f.write(f"- {topic}\n")  # Menulis topik ke file

    print(f"Processed: {file_path} -> {file_output_folder}")  # Menampilkan pesan bahwa file telah diproses

def main():
    # Fungsi utama untuk menjalankan proses
    input_folder = "data"  # Menentukan folder input
    output_folder = "output/graph-topic-rank"  # Menentukan folder output
    os.makedirs(output_folder, exist_ok=True)  # Membuat folder output jika belum ada
    
    for file_name in os.listdir(input_folder):  # Iterasi melalui setiap file dalam folder input
        file_path = os.path.join(input_folder, file_name)  # Mendapatkan jalur lengkap file
        if os.path.isfile(file_path):  # Memastikan bahwa jalur adalah file
            process_document(file_path, output_folder)  # Memproses dokumen

if __name__ == "__main__":
    main()  # Menjalankan fungsi utama saat skrip dieksekusi

Processed: data\dokumen_pdf.pdf -> output/graph-topic-rank\dokumen_pdf
Processed: data\tko.pdf -> output/graph-topic-rank\tko


#### ADVANCED SEMANTIC CHUNKING - LLAMA WITH EMBEDDING RANK

Metode ini menggunakan pemrosesan bahasa alami (NLP) berbasis spaCy untuk chunking teks, serta peringkat berbasis embedding menggunakan model `HuggingFaceEmbeddings`. Model LDA (Latent Dirichlet Allocation) digunakan untuk mengekstrak topik utama dari setiap chunk.

Cara Kerja:
- Dokumen dipecah menjadi chunk menggunakan model NLP spaCy.
- Embedding setiap chunk dihitung menggunakan `HuggingFaceEmbeddings`.
- Chunk diberi peringkat berdasarkan norma embedding menggunakan `numpy.linalg.norm`.
- Topik utama dari setiap chunk diekstrak menggunakan Gensim LDA.
- Topik terbaik dipilih berdasarkan probabilitas tertinggi dalam distribusi topik chunk tersebut.

Kelebihan:
- Menggunakan embedding untuk peringkat yang lebih akurat dibanding metode berbasis kata.
- Mampu mengidentifikasi topik utama dalam setiap chunk dengan metode LDA.
- Dapat diterapkan pada berbagai jenis dokumen (TXT, PDF, DOCX).

Kekurangan:
- LDA masih berbasis bag-of-words, sehingga tidak mempertimbangkan urutan kata.
- Kualitas chunk tergantung pada model NLP yang digunakan.
- Hasil topik bisa kurang akurat jika jumlah topik tidak ditentukan dengan baik.

In [ ]:
import os  # Mengimpor modul os untuk operasi sistem file
import spacy  # Mengimpor spaCy untuk pemrosesan bahasa alami
import docx  # Mengimpor modul docx untuk membaca file DOCX
import fitz  # Mengimpor fitz (PyMuPDF) untuk membaca file PDF
import gensim  # Mengimpor gensim untuk model topik
from gensim import corpora  # Mengimpor corpora dari gensim untuk membuat kamus
from langchain_huggingface import HuggingFaceEmbeddings  # Mengimpor model embedding dari Hugging Face
from langchain_ollama.llms import OllamaLLM  # Mengimpor model LLM dari Ollama

# Memuat model bahasa Inggris kecil dari spaCy
nlp = spacy.load("en_core_web_sm")
OLLAMA_MODEL = "llama3.2"  # Menentukan model Ollama yang akan digunakan
llm = OllamaLLM(model=OLLAMA_MODEL)  # Memuat model LLM
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")  # Memuat model embedding

def read_txt(file_path):
    # Membaca file teks dan mengembalikan isinya
    with open(file_path, "r", encoding="utf-8") as f:  # Membuka file dalam mode baca
        return f.read()  # Mengembalikan seluruh isi file

def read_pdf(file_path):
    # Membaca file PDF dan mengembalikan isinya
    text = ""  # Inisialisasi string kosong untuk menyimpan teks
    with fitz.open(file_path) as doc:  # Membuka file PDF menggunakan fitz
        for page in doc:  # Iterasi melalui setiap halaman
            text += page.get_text("text") + "\n"  # Ekstrak teks dari halaman dan tambahkan ke string
    return text  # Mengembalikan teks yang diekstrak

def read_docx(file_path):
    # Membaca file DOCX dan mengembalikan isinya
    doc = docx.Document(file_path)  # Membuka file DOCX
    return "\n".join([para.text for para in doc.paragraphs])  # Menggabungkan teks dari setiap paragraf

def clean_text(text):
    # Membersihkan teks dengan menghapus newline dan menghapus spasi berlebih
    text = text.replace("\n", " ").strip()  # Mengganti newline dengan spasi dan menghapus spasi di awal/akhir
    text = " ".join(text.split())  # Menghapus spasi berlebih di antara kata
    return text  # Mengembalikan teks yang telah dibersihkan

def semantic_chunking(text, chunk_size=700):
    # Membagi teks menjadi bagian-bagian kecil berdasarkan ukuran yang ditentukan
    doc = nlp(text)  # Memproses teks dengan spaCy
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]  # Mengambil kalimat yang tidak kosong
    
    if not sentences:  # Jika tidak ada kalimat yang valid
        return []  # Mengembalikan daftar kosong
    
    chunks = []  # Inisialisasi daftar untuk menyimpan bagian
    chunk = ""  # Inisialisasi string untuk bagian saat ini
    for sentence in sentences:  # Iterasi melalui setiap kalimat
        if len(chunk) + len(sentence) < chunk_size:  # Jika ukuran bagian saat ini ditambah kalimat masih dalam batas chunk_size
            chunk += " " + sentence  # Tambahkan kalimat ke bagian saat ini
        else:
            chunks.append(chunk.strip())  # Tambahkan bagian saat ini ke daftar
            chunk = sentence  # Mulai bagian baru dengan kalimat saat ini
    if chunk:  # Jika ada bagian yang tersisa
        chunks.append(chunk.strip())  # Tambahkan bagian terakhir ke daftar

    return chunks  # Mengembalikan daftar bagian

def extract_topics(text, num_topics=3):
    # Mengekstrak topik dari teks menggunakan model LDA
    words = [token.lemma_ for token in nlp(text) if token.is_alpha and not token.is_stop]  # Tokenisasi dan lemmatization

    if not words:  # Jika tidak ada kata yang valid
        return ["[No valid words for topic extraction]"]  # Mengembalikan pesan bahwa tidak ada kata valid

    dictionary = corpora.Dictionary([words])  # Membuat kamus dari kata-kata
    corpus = [dictionary.doc2bow(words)]  # Membuat korpus dari kata-kata

    if not corpus or all(len(doc) == 0 for doc in corpus):  # Jika korpus kosong
        return ["[No significant topics]"]  # Mengembalikan pesan bahwa tidak ada topik yang signifikan

    lda_model = gensim.models.LdaMulticore(  # Membuat model LDA dengan dukungan multi-core
        corpus, num_topics=min(num_topics, len(dictionary)),  # Menentukan jumlah topik yang akan diekstrak
        id2word=dictionary, passes=10, workers=4  # Mengatur parameter model LDA
    )
    topics = lda_model.show_topics(formatted=False)  # Mengambil topik yang dihasilkan oleh model

    return [", ".join([word for word, _ in topic[1]]) for topic in topics]  # Mengembalikan daftar topik yang diekstrak

def process_document(file_path, output_folder):
    # Memproses dokumen berdasarkan jenis file dan menyimpan hasilnya
    ext = file_path.split(".")[-1].lower()  # Mendapatkan ekstensi file
    
    # Membaca teks berdasarkan jenis file
    if ext == "txt":
        text = read_txt(file_path)  # Membaca file teks
    elif ext == "pdf":
        text = read_pdf(file_path)  # Membaca file PDF
    elif ext == "docx":
        text = read_docx(file_path)  # Membaca file DOCX
    else:
        print(f"Unsupported file format: {ext}")  # Menampilkan pesan jika format file tidak didukung
        return  # Keluar dari fungsi
    
    base_name = os.path.basename(file_path).split(".")[0]  # Mendapatkan nama dasar file tanpa ekstensi
    file_output_folder = os.path.join(output_folder, base_name)  # Menentukan folder output untuk file ini
    os.makedirs(file_output_folder, exist_ok=True)  # Membuat folder output jika belum ada

    cleaned_text = clean_text(text)  # Membersihkan teks dari file
    chunks = semantic_chunking(cleaned_text)  # Membagi teks menjadi bagian-bagian kecil
    
    for i, chunk in enumerate(chunks):  # Iterasi melalui setiap bagian
        chunk_file = os.path.join(file_output_folder, f"{base_name}_chunk_{i+1}.txt")  # Menentukan nama file untuk bagian
        topics = extract_topics(chunk)  # Mengekstrak topik dari bagian
        
        with open(chunk_file, "w", encoding="utf-8") as f:  # Membuka file untuk menulis
            f.write(chunk + "\n\n")  # Menulis bagian ke file
            f.write("Topik:\n")  # Menulis header untuk topik
            for topic in topics:  # Iterasi melalui setiap topik
                f.write(f"- {topic}\n")  # Menulis topik ke file

    print(f"Processed: {file_path} -> {file_output_folder}")  # Menampilkan pesan bahwa file telah diproses

def main():
    # Fungsi utama untuk menjalankan proses
    input_folder = "data"  # Menentukan folder input
    output_folder = "output/llm-semantic-chunking"  # Menentukan folder output
    os.makedirs(output_folder, exist_ok=True)  # Membuat folder output jika belum ada
    
    for file_name in os.listdir(input_folder):  # Iterasi melalui setiap file dalam folder input
        file_path = os.path.join(input_folder, file_name)  # Mendapatkan jalur lengkap file
        if os.path.isfile(file_path):  # Memastikan bahwa jalur adalah file
            process_document(file_path, output_folder)  # Memproses dokumen

if __name__ == "__main__":
    main()  # Menjalankan fungsi utama saat skrip dieksekusi

Processed: data\dokumen_pdf.pdf -> output/llm-semantic-chunking\dokumen_pdf
Processed: data\tko.pdf -> output/llm-semantic-chunking\tko


MODELING

In [14]:
import os  # Mengimpor modul os untuk operasi sistem file
from IPython.display import Markdown  # Mengimpor Markdown untuk menampilkan teks dalam format Markdown di Jupyter Notebook
from langchain_core.prompts import ChatPromptTemplate  # Mengimpor ChatPromptTemplate untuk membuat template prompt
from langchain_ollama.llms import OllamaLLM  # Mengimpor OllamaLLM untuk menggunakan model LLM

OLLAMA_MODEL = "llama3.2"  # Menentukan model LLM yang akan digunakan

# Memuat model LLM
model = OllamaLLM(model=OLLAMA_MODEL)  # Menginisialisasi model LLM dengan nama model yang ditentukan

# Template prompt untuk ringkasan
template = """
1. Apa tujuan dari tata kerja organisasi?
2. Apa pengertian Onsite Support dan Aplikasi Upstream?
3. Apa dokumen dan referensi terkait Tata Kerja Organisasi?
4. Bagaimana prosedur permintaan baru akun aplikasi?
Gunakan hanya informasi yang terdapat dalam dokumen.  

Dokumen:
"{document}"
"""  # Template yang akan digunakan untuk meminta ringkasan dari model LLM

# Membuat objek ChatPromptTemplate dari template yang telah ditentukan
prompt = ChatPromptTemplate.from_template(template)

def summarize_text(text):
    chain = prompt | model  # Menggabungkan prompt dengan model LLM untuk membuat rantai pemrosesan
    response = chain.invoke({"document": text})  # Mengirimkan teks ke model LLM untuk diringkas
    return response  # Mengembalikan hasil ringkasan

def read_all_chunks(folder):
    all_text = []  # Inisialisasi daftar untuk menyimpan isi semua file
    
    # Ambil semua file dalam folder yang dimulai dengan "chunk_" dan diakhiri dengan ".txt", lalu urutkan berdasarkan nama file
    files = sorted([f for f in os.listdir(folder) if f.startswith("chunk_") and f.endswith(".txt")])
    
    for file in files:  # Iterasi melalui setiap file yang ditemukan
        file_path = os.path.join(folder, file)  # Mendapatkan jalur lengkap file
        with open(file_path, "r", encoding="utf-8") as f:  # Membuka file dalam mode baca
            all_text.append(f.read())  # Tambahkan isi file ke daftar
    
    return "\n\n".join(all_text)  # Gabungkan semua isi file dengan pemisah newline dan kembalikan sebagai satu string

# Baca semua chunk dari folder
output_folder = "output/llm-semantic-chunking/tko"  # Menentukan folder output yang berisi file chunk
document_text = read_all_chunks(output_folder)  # Membaca semua chunk dan menggabungkan isinya

# Buat ringkasan
summary = summarize_text(document_text)  # Menggunakan fungsi untuk meringkas teks yang telah dibaca

# Tampilkan hasil sebagai Markdown
Markdown(summary)  # Menampilkan ringkasan dalam format Markdown di Jupyter Notebook

Berikut adalah jawaban mengenai pertanyaan Anda berdasarkan informasi yang tersedia dalam dokumen:

1. Tujuan dari tata kerja organisasi adalah untuk menentukan proses dan struktur kerja yang efektif dan efisien dalam mencapai tujuan perusahaan.
2. Onsite Support adalah layanan teknis yang menyediakan dukungan langsung untuk aplikasi, sedangkan Aplikasi Upstream adalah aplikasi yang dikembangkan dan diterbitkan oleh tim pengembang di luar organisasi yang menerimanya.
3. Dokumen terkait Tata Kerja Organisasi dapat ditemukan dalam bagian "Referensi" pada dokumen ini, yaitu:
 * Tata Kerja Organisasi
 * Prosedur Penggunaan Aplikasi
 * Standar Komunikasi Tim
4. Procedur permintaan baru akun aplikasi adalah sebagai berikut:
 * Langkah 1: Isi formulir permintaan baru akun aplikasi yang tersedia pada dokumen ini.
 * Langkah 2: Klik tombol "Submit" untuk mengirimkan permintaan.
 * Langkah 3: Tim pengembang akan memproses permintaan dan melakukan verifikasi bahwa akun tersebut dapat digunakan dengan aman.
 * Langkah 4: Jika permintaan diverifikasi, akun baru akan diaktifkan dan pengguna akan mendapatkan notifikasi.

Perlu diingat bahwa prosedur ini mungkin berbeda-beda tergantung pada kebutuhan dan kebijakan organisasi.